Run this below script to get the query result, added only one query for the time being, will add more later

In [ ]:
from fastapi import FastAPI
import snowflake.connector
import pandas as pd
from pydantic import BaseModel
from typing import List

# Create FastAPI instance
app = FastAPI()

# Snowflake connection configuration
config = {
    "account": "SFEDU02-KFB85562",
    "user": "FOX",
    "password": "Arjun9481@",
    "role": "TRAINING_ROLE",
    "warehouse": "ANIMAL_TASK_WH",
    "database": "CATCH_ME_IF_YOU_CAN",
    "schema": "PUBLIC"
}

# Define response model for type safety
class TableUsage(BaseModel):
    table_id: str
    total_count: int
    join_count: int
    join_usage_ratio: float

@app.get("/table-usage", response_model=List[TableUsage])
async def get_table_usage():
    try:
        # Connect to Snowflake
        conn = snowflake.connector.connect(**config)
        cur = conn.cursor()

        # SQL Query
        query = """
        WITH exploded_tables AS (
            SELECT
                query_id,
                num_joins,
                t.value AS table_id
            FROM
                live_queries,
                LATERAL FLATTEN(INPUT => SPLIT(read_table_ids, ',')) t
            WHERE
                read_table_ids IS NOT NULL
        ),
        join_usage AS (
            SELECT
                table_id,
                COUNT(*) AS join_count
            FROM
                exploded_tables
            WHERE
                num_joins > 0
            GROUP BY
                table_id
        ),
        total_usage AS (
            SELECT
                table_id,
                COUNT(*) AS total_count
            FROM
                exploded_tables
            GROUP BY
                table_id
        ),
        usage_analysis AS (
            SELECT
                t.table_id,
                t.total_count,
                j.join_count,
                (j.join_count * 1.0 / t.total_count) AS join_usage_ratio
            FROM
                total_usage t
            LEFT JOIN
                join_usage j
            ON
                t.table_id = j.table_id
        )
        SELECT
            table_id,
            total_count,
            join_count,
            join_usage_ratio
        FROM
            usage_analysis
        WHERE
            join_usage_ratio = 1.0
        ORDER BY
            total_count DESC;
        """

        # Execute the query and fetch results
        cur.execute(query)
        df = cur.fetch_pandas_all()

        # Convert DataFrame to a list of dictionaries
        results = df.to_dict(orient="records")

        return results

    except Exception as e:
        return {"error": str(e)}

    finally:
        # Close the connection
        if 'cur' in locals():
            cur.close()
        if 'conn' in locals():
            conn.close()


Run this to host the local server for the API running

In [ ]:
import uvicorn
# Start the FastAPI app in the notebook
uvicorn.run("main:app", host="127.0.0.1", port=8000, reload=True)